## 8/2/23
* We start to reformulate this code from the original in magma for g4
*

## 15/2/23 
* Continue qith the formulation

## 23/2/23
* I started redoing this code, I thought we could change to finding eigenvectors, but there might happen that the point is ont one of the two generators but a linear conbimation of them, so it is best to continue with the method of creating a scheme with the linear polynomials coming from the matrix and intersectiong with the cubic. 

## 19/5/23
* I finalized this code

In [ ]:
##This code assumes we have
def hasse_witt_matrix_g3(tup): #The input is a 15-tuple with the coefficients of the quartic
    SS.<X,Y,Z,W>=PolynomialRing(K,4)
       
    mon_degree_4xyz=[X^4,X^3*Y,X^3*Z,X^2*Y^2,X^2*Y*Z,X^2*Z^2,X*Y^3,X*Y^2*Z,X*Y*Z^2,X*Z^3,Y^4,
                        Y^3*Z,Y^2*Z^2,Y*Z^3,Z^4]
    G=sum([tup[i]*mon_degree_4xyz[i] for i in range(15)])
    HW_expon=[[2*p-2,p-1,p-1,p-1],[p-2,2*p-1,p-1,p-1],[p-2,p-1,2*p-1,p-1],
              [2*p-1,p-2,p-1,p-1],[p-1,2*p-2,p-1,p-1],[p-1,p-2,2*p-1,p-1],
              [2*p-1,p-1,p-2,p-1],[p-1,2*p-1,p-2,p-1],[p-1,p-1,2*p-2,p-1]];
 
    HW_mono=[X^(E[0])*Y^(E[1])*Z^(E[2])*W^(E[3]) for E in HW_expon]
    

    H=(W*G)^(p-1);
    HW_coeff=[H.monomial_coefficient(m) for m in HW_mono]
    HW_matrix=matrix(3,3,HW_coeff)
    
    return HW_matrix

In [ ]:
def List_of_T2CP(curve): #Input is a curve in our list curve=(tuple, p-rank,a-number)
    tup=curve[0]
    f=curve[1]
    ListT2CP=[[],[],[],[]]
    if f==0:
        ListT2CP=[[],[],[],[]]

    else:
        PP2.<x,y,z>=ProjectiveSpace(K,2)
        quart=sum([tup[i]*mon_degree_4xyz[i] for i in range(15)])
        HW1=hasse_witt_matrix_g3(tup)

        if f==1: ##If f=1 there will only be one nonzero eigenvalue with geometric multiplicity 1, 
            #so we only need to look at an eigenvector and check if it is a point over the curve
            eig = HW1.eigenvectors_right()
            nonzero_eig = [e[0] for e in eig if e[0] != 0]
            nonozero_eigenvectors = [e[1][0] for e in eig if e[0] in nonzero_eig]
            v=nonozero_eigenvectors[0]
            result=quart.subs(x=v[0],y=v[1],z=v[2])
            if result==0:
                ListT2CP[0]=[v]

        elif f==2:
            px=HW1.characteristic_polynomial()
            Kd.<a>=px.splitting_field()
            degKd=Kd.degree()
            HW1=MatrixSpace(Kd,3,3)(HW1) #Emded HW into the splitting field
            eig = HW1.eigenvectors_right()
            nonzero_eig = [e for e in eig if e[0] != 0]

            if len(nonzero_eig)==1: #If there is only one nozero eigenvalue *it must be over Fp 
                if nonzero_eig[0][2]==1: #If the dimension of the eigenspace is 1
                    v=nonzero_eig[0][2] #v is an eigenvalue in the eigenspace
                    result=quart.subs(x=v[0],y=v[1],z=v[2])
                    if result==0:
                        ListT2CP[0]=[v]

                elif nonzero_eig[0][2]==2: #If the dimension of the eigenspace is 2
                    a1=nonzero_eig[0][0]
                    poly1=HW1[0][0]*x+HW1[0][1]*y+HW1[0][2]*z-a1*x
                    poly2=HW1[1][0]*x+HW1[1][1]*y+HW1[1][2]*z-a1*y
                    poly3=HW1[2][0]*x+HW1[2][1]*y+HW1[2][2]*z-a1*z
                    polys=[quart, poly1,poly2,poly3]#This is the set of polynomials that form the scheme 

                    V=ProjectiveSpace(Kd,2).subscheme(polys)
                    CPs_1=[[P[0],P[1],P[2]] for P in V.rational_points()]#the CPs of degree 1
                    ListT2CP[0]=CPs_1


                # ListT2CP=[[],[],[],[]]
                    for i in [2,3,4]:
                        Intersection_i=V.base_extend(Kd.extension(i))
                        CPs_i=[[P[0],P[1],P[2]] for P in Intersection_i.rational_points()]
                        for j in [1..i-1]:
                            CPs_i=[v for v in CPs_i if not [v[0]^(p^j),v[1]^(p^j),v[2]^(p^j)]==v]#This removes repeated 
                        ListT2CP[i-1]=CPs_i



            elif len(nonzero_eig)==2:
                eig = HW1.eigenvectors_right()
                nonzero_eig = [e[0] for e in eig if e[0] != 0]
                nonozero_eigenvectors = [e[1][0] for e in eig if e[0] in nonzero_eig]
                for v in nonozero_eigenvectors: #v is an eigenvalue in the eigenspace
                    result=quart.subs(x=v[0],y=v[1],z=v[2])
                    if result==0:
                        ListT2CP[0]=[v]+ListT2CP[0]

    return ListT2CP

In [ ]:
def All_T2_points_complete_by_degree_g3(curve):
    All_T2points=List_of_T2CP(curve)
    Sum_and_degree=[len(plist) for plist in All_T2points]
    return [All_T2points, Sum_and_degree]